## Based on Code_for_Spring_2026_sched

In [1]:
from docplex.mp.model import Model
import pandas as pd
from pandas import DataFrame
import datetime
#import xlrd
import matplotlib.pyplot as plt 
import numpy as np
import re
import itertools
import math
from datetime import datetime
#loading packages that are needed
# TPS 
from docplex.mp.conflict_refiner import ConflictRefiner
from docplex.mp.relaxer import Relaxer
#This isn't where the infeasibility notebook imports priority from, which is vaguely concerning
from docplex.mp.priority import Priority

# New in Nov 2025: Progress Listeners
# Possibly get rid of the "*" in the future
from docplex.mp.progress import *

### Read in homemade functions from .py files

In [2]:
from constraint_dist_throughout_day_Oct25 import constraint_dist_throughout_day
from constraint_each_class_taught_correct_number_of_times_Oct25 import constraint_each_class_taught_correct_number_of_times
from constraint_prof_num_credits_Oct25 import constraint_prof_num_credits

# Below this is new in Nov 2025
from constraint_dist_throughout_day_Oct25 import constraint_dist_throughout_day
from constraint_each_class_taught_correct_number_of_times_Oct25 import constraint_each_class_taught_correct_number_of_times
from constraint_prof_num_credits_Oct25 import constraint_prof_num_credits
from constraint_no_overlapping_classes_Oct25 import constraint_no_overlapping_classes
from constraint_course_course_time_avoid_and_require_Oct25 import constraint_course_course_time_avoid
from constraint_course_course_time_avoid_and_require_Oct25 import constraint_course_course_time_require
from constraint_course_time_Oct25 import constraint_course_time
from constraint_course_course_prof_avoid_and_require_Oct25 import constraint_course_course_prof_avoid_and_require
# for now, not using preference_dist_throughtout_day_actual_Oct25.py

### Small change compared to Feb_2025:
July 2025:  Split the info from Qualtrics into a separate file than the goal nums.  Added variables to control which preferences, constraints, and method of dealing with b2b courses we use.

In [3]:
# file names
# Using Overlapping_times_version_4.ipynb that automatically renames the output file in a particular way
# True/False flag to rename the overlaps for you
time_option_overlaps_auto = True 
prev_schedule = False # true/false flag to see if needing to run previous schedule stuff
qualtrics_file_short = 'Professor_Input_for_GitHub_small'
#'Spring2026_Medium_Professor_Input_data_Nov_25' 
qualtrics_file_name = f'{qualtrics_file_short}.xlsx'
targets_file_name = 'Targets_for_GitHub_small.xlsx' #'Spring_2026_to_merge_with_Qualtrics_with_Cannot_teach_update_credits.xlsx'
time_groups_file_name = 'Meeting_Patterns_for_GitHub_small.xlsx'#'Meeting_Patterns_small_Oct_2024.xlsx' #'Meeting_Patterns_Medium_Nov_2024_3.xlsx'
time_option_file_name = ''
if time_option_overlaps_auto:
    time_option_file_name = time_groups_file_name.removesuffix('.xlsx') + '_overlaps.xlsx'
    
##### For testing b2b course code's effects on memory usage
b2b_Theodore_Feb_2025 = False
b2b_Fall_2025_sched = not b2b_Theodore_Feb_2025
listen_progress = True # New in Nov 2025.  Do you want a Progress Listener?

use_course_course_prof = False # if False, ignores preferences and constraints involving similar courses



# Theodore 21 Feb 2025
# Moved this to the bottom of the notebook, 7 Mar 2025
##output_file_name = f'results_{class_file_short}.csv'
# now = datetime.now()
# today_str = now.strftime("%Y:%m:%d_%H:%M:%S")
# output_file_name = f'results_{class_file_short}_{today_str}.csv'


if prev_schedule:
    partial_file =  f'{qualtrics_file_short}_results_partial.csv' #partial results to use when schedule is premade
    output_file_name = f'{qualtrics_file_short}_results_partial.csv' # Where the resulting schedule will be stored

In [4]:
# constants for the model
max_number_sections_to_be_at_diff_times = 3 # if a course has this number or less sections, they must happen at different times, as in adding a constraint for these courses to not overlap irregardless of professor
pref_scaler = 2.5

## Change compared to Fall_2025_schedule.ipynb:

course_course_time_prefs is commented out because we are moving those preferences into low-priority constraints, to avoid "non-convex objective" error.

### Small change compared to Feb_2025:
Changing which file some of the data is read from.

In [5]:
# converting files/sheets into df
class_xls_file = pd.ExcelFile(qualtrics_file_name)
time_xls_file = pd.ExcelFile(time_option_file_name)
targets_xls_file = pd.ExcelFile(targets_file_name)
time_group_xls_file = pd.ExcelFile(time_groups_file_name)
parseable_sheets = class_xls_file.sheet_names + time_xls_file.sheet_names + time_group_xls_file.sheet_names + targets_xls_file.sheet_names
did_not_parse = []
if prev_schedule:
    partial_results_file = pd.read_csv(partial_file)
    partial_results_file["all_times"]=partial_results_file["all_times"].convert_dtypes(int)
df_preferences = class_xls_file.parse('Preferences')
df_goals = targets_xls_file.parse('Goal_nums')
df_credits = targets_xls_file.parse('class_credits')
df_times = time_xls_file.parse()
df_time_groups = time_group_xls_file.parse('Time_Group_Info')
df_time_pref = class_xls_file.parse("Time_preferences")
df_credits_per_prof = targets_xls_file.parse("Credits_per_prof")
df_back_to_back_pref = class_xls_file.parse("Back_to_back")
df_preps_pref = class_xls_file.parse("Preps") #adding df of preps preferences
if 'Course_course_prof_prefs' in parseable_sheets:
    df_course_course_prof_prefs = time_group_xls_file.parse('Course_course_prof_prefs')
else:
    did_not_parse.append('Course_course_prof_prefs')
if 'Course_course_prof_constraints' in parseable_sheets:
    df_course_course_prof_constraints = time_group_xls_file.parse('Course_course_prof_constraints')
else:
    did_not_parse.append('Course_course_prof_constraints')
# if 'Course_course_time_prefs' in parseable_sheets:
#     df_course_course_time_prefs = time_group_xls_file.parse('Course_course_time_prefs')
# else:
#     did_not_parse.append('Course_course_time_prefs')
if 'Course_course_time_constraints'  in parseable_sheets:   
    df_course_course_time_constraints = time_group_xls_file.parse('Course_course_time_constraints')
else:
    did_not_parse.append('Course_course_time_constraints')
if 'Course_time_prefs' in parseable_sheets:
    df_course_time_prefs = time_group_xls_file.parse('Course_time_prefs')
else:
    did_not_parse.append('Course_time_prefs')
if 'Course_time_constraints' in parseable_sheets:
    df_course_time_constraints = time_group_xls_file.parse('Course_time_constraints')
else:
    did_not_parse.append('Course_time_constraints')
if 'Cannot_teach' in parseable_sheets:
    df_prof_cannot_teach = targets_xls_file.parse("Cannot_teach") # TPS
else:
    did_not_parse.append('Cannot_teach')
print(f'{did_not_parse=}')

did_not_parse=[]


In [6]:
#df_course_course_time_constraints

## Tell me which classes (if any) are missing credit numbers
### (New in Fall_2025_schedule.ipynb)
From among courses with Goal_max > 0 

In [7]:
credit_check = pd.merge(df_goals, df_credits, how="left", left_on="Course", right_on="course_name")
credit_check = credit_check[credit_check["Goal_number_max"] > 0]
credit_check = credit_check[credit_check["number_of_credits"].isna()]
if credit_check.shape[0] > 0:
    raise ValueError("class_credits sheet does not include the following courses:\n" + credit_check["Course"].to_string())
#df_credits

## Check for missing credit numbers for faculty
### Small change from Feb_2025:  Removes faculty with max_credits = 0 from df_credits_per_prof, so it no longer complains if a faculty member with max_credits = 0 has missing preferences.

In [8]:
df_credits_per_prof = df_credits_per_prof.loc[df_credits_per_prof["num_credits_max"] > 0,:]
prof_check = pd.merge(df_preferences, df_credits_per_prof, how="outer", left_on="Instructor", right_on="prof_name")
missing_prefs = prof_check.loc[prof_check["Instructor"].isna(), "prof_name"]
missing_prof_credits = prof_check.loc[prof_check["prof_name"].isna(), "Instructor"]
if missing_prefs.shape[0] > 0:
    raise ValueError("No preferences available for the following instructors:\n" + missing_prefs.to_string())
if missing_prof_credits.shape[0] > 0:
    raise ValueError("No min/max credit numbers for the following instructors:\n" + missing_prof_credits.to_string())


### New in July 2025: 
Also check spelling of professors' names in Time_preferences, Back-to-back, and Preps.  Normally not an issue, because Parse_Qualtrics sets the names to match.  But if I forgot to fix a spelling issue in the original download from Qualtrics and then try to change it after running Parse_Qualtrics, this code cell will alert me if I missed one of the sheets.

In [9]:
prof_time_check = pd.merge(df_credits_per_prof, df_time_pref, how="outer", left_on="prof_name", right_on="Prof_name")
missing_time_prefs = prof_time_check.loc[prof_time_check["Prof_name"].isna(), "prof_name"]
extra_time_prof = prof_time_check.loc[prof_time_check["prof_name"].isna(), "Prof_name"]
if missing_time_prefs.shape[0] > 0:
    raise ValueError("No time preferences available for the following instructors:\n" + missing_time_prefs.to_string())
if extra_time_prof.shape[0] > 0:
    raise ValueError("The following instructors have time preferences but no min/max credit numbers:\n" + extra_time_prof.to_string())
    
prof_b2b_check = pd.merge(df_credits_per_prof, df_back_to_back_pref, how="outer", left_on="prof_name", right_on="Prof_name")
missing_b2b_prefs = prof_b2b_check.loc[prof_b2b_check["Prof_name"].isna(), "prof_name"]
extra_b2b_prof = prof_b2b_check.loc[prof_b2b_check["prof_name"].isna(), "Prof_name"]
if missing_b2b_prefs.shape[0] > 0:
    raise ValueError("No back-to-back course preferences available for the following instructors:\n" + missing_b2b_prefs.to_string())
if extra_b2b_prof.shape[0] > 0:
    raise ValueError("The following instructors have back-to-back course preferences but no min/max credit numbers:\n" + extra_b2b_prof.to_string())

prof_preps_check = pd.merge(df_credits_per_prof, df_preps_pref, how="outer", left_on="prof_name", right_on="Prof_name")
missing_preps_prefs = prof_preps_check.loc[prof_preps_check["Prof_name"].isna(), "prof_name"]
extra_preps_prof = prof_preps_check.loc[prof_preps_check["prof_name"].isna(), "Prof_name"]
if missing_preps_prefs.shape[0] > 0:
    raise ValueError("No preferences about number of preps available for the following instructors:\n" + missing_preps_prefs.to_string())
if extra_preps_prof.shape[0] > 0:
    raise ValueError("The following instructors have preferences about number of preps but no min/max credit numbers:\n" + extra_preps_prof.to_string())


In [10]:
#df_preps_pref

## Check that all courses with Goal_max > 0 have at least one possible meeting time
### Small change from Feb_2025:
added on="Time_Group" to the merge, to prevent Time_Group M from disappearing (due to a mismatch in the Notes columns of the two data frames, I think)

In [11]:
# First check that all courses with Goal_max > 0 have (at least one) defined Time Group
meeting_time_check = pd.merge(df_goals, df_time_groups, how="left", left_on="Course", right_on="Class")
meeting_time_check = meeting_time_check.loc[meeting_time_check["Goal_number_max"] > 0, ]
not_listed = meeting_time_check.loc[meeting_time_check["Time_Group"].isna(), "Course"]
if not_listed.shape[0] > 0:
    raise ValueError("Time Groups not defined for the following courses:\n" + not_listed.to_string())

# Then check that all of those Time Groups have (at least one) meeting time
time_slots_for_time_groups = pd.merge(meeting_time_check, df_times, how="left", on="Time_Group")
# time_slots_for_time_groups[["Course", "Time_Index"]]
# no_time contains courses with an NaN Time Index.  But they might also have a valid time index elsewhere in the dataframe.
no_time = time_slots_for_time_groups.loc[time_slots_for_time_groups["Time_Index"].isna(), "Course"] 
has_time = time_slots_for_time_groups.loc[time_slots_for_time_groups["Time_Index"].notna(), "Course"]
# Courses that *only* have an NaN Time Index
really_no_time = set(no_time).difference(set(has_time))
if len(really_no_time) > 0:
    raise ValueError("The following courses have no meeting times (not even asynchronous):\n" + ", ".join(really_no_time))

## formatting the dfs to work well with each other and the model

In [12]:
df_credits_per_prof = df_credits_per_prof.loc[df_credits_per_prof['num_credits_max'] > 0]
df_credits_per_prof.rename(columns = {'prof_name' : 'all_profs'}, inplace=True)
profs_with_enough_creds = df_credits_per_prof['all_profs'].tolist()
#df_credits_per_prof

In [13]:
df_goals.rename(columns = {'Course' : 'all_courses'}, inplace=True)
df_goals = df_goals.loc[df_goals['Goal_number_max'] > 0]
courses_with_enough_sections = df_goals['all_courses'].tolist()
#df_goals

In [14]:
df_preferences.rename(columns = {'Instructor' : 'all_profs'}, inplace=True)
courses_with_enough_sections_and_other_headers = courses_with_enough_sections.append("all_profs")
df_preferences = df_preferences.loc[df_preferences['all_profs'].isin(profs_with_enough_creds),df_preferences.columns.isin(courses_with_enough_sections)].copy()
#df_preferences

In [15]:
df_credits.rename(columns = {'course_name' : 'all_courses'}, inplace=True)
df_credits = df_credits.loc[df_credits['all_courses'].isin(courses_with_enough_sections)].copy()
#df_credits

In [16]:
df_times = df_times.loc[:, ~df_times.columns.str.contains('^Unnamed')]
df_times.rename(columns = {'Time_Index' : 'all_times'}, inplace=True)
df_times.rename(columns = {'Days' : 'Day'}, inplace=True)
df_times['all_times'] = df_times['all_times'].astype(int)
#df_times

In [17]:
df_time_pref.rename(columns = {'Prof_name' : 'all_profs'}, inplace=True)
df_time_pref = df_time_pref.loc[df_time_pref['all_profs'].isin(profs_with_enough_creds)].copy()
# df_time_pref

In [18]:
df_back_to_back_pref.rename(columns = {"Prof_name":"all_profs"}, inplace=True)
df_back_to_back_pref = df_back_to_back_pref.loc[df_back_to_back_pref['all_profs'].isin(profs_with_enough_creds)].copy()
# df_back_to_back_pref

In [19]:
df_course_course_prof_prefs.drop(columns= 'Notes', inplace = True)
df_course_course_prof_prefs = df_course_course_prof_prefs.loc[df_course_course_prof_prefs['Course_1'].isin(courses_with_enough_sections)]
df_course_course_prof_prefs = df_course_course_prof_prefs.loc[df_course_course_prof_prefs['Course_2'].isin(courses_with_enough_sections)]
# df_course_course_prof_prefs

In [20]:
df_course_course_prof_constraints.drop(columns= 'Notes', inplace = True)
df_course_course_prof_constraints = df_course_course_prof_constraints.loc[df_course_course_prof_constraints['Course_1'].isin(courses_with_enough_sections)]
df_course_course_prof_constraints = df_course_course_prof_constraints.loc[df_course_course_prof_constraints['Course_2'].isin(courses_with_enough_sections)]
# df_course_course_prof_constraints

In [21]:
# Change compared to Fall_2025_schedule.ipynb:  
# Commented this cell out because we are moving course_course_time_prefs into low-priority constraints.

# if 'Notes' in df_course_course_time_prefs.columns:
#     df_course_course_time_prefs.drop(columns = 'Notes', inplace = True)
# df_course_course_time_prefs = df_course_course_time_prefs.loc[df_course_course_time_prefs['Course_1'].isin(courses_with_enough_sections)]
# df_course_course_time_prefs = df_course_course_time_prefs.loc[df_course_course_time_prefs['Course_2'].isin(courses_with_enough_sections)]
# # df_course_course_time_prefs

In [22]:
df_course_course_time_constraints.drop(columns = 'Notes', inplace = True)
df_course_course_time_constraints = df_course_course_time_constraints.loc[df_course_course_time_constraints['Course_1'].isin(courses_with_enough_sections)]
df_course_course_time_constraints = df_course_course_time_constraints.loc[df_course_course_time_constraints['Course_2'].isin(courses_with_enough_sections)]
# df_course_course_time_constraints

In [23]:
df_course_time_prefs.drop(columns = 'Notes', inplace = True)
df_course_time_prefs = df_course_time_prefs.loc[df_course_time_prefs['Course'].isin(courses_with_enough_sections)]
# df_course_time_prefs

In [24]:
df_course_time_constraints.drop(columns = 'Notes', inplace = True)
df_course_time_constraints = df_course_time_constraints.loc[df_course_time_constraints['Course'].isin(courses_with_enough_sections)]
# df_course_time_constraints

In [25]:
df_prof_cannot_teach.drop(columns='Notes', inplace=True)
df_prof_cannot_teach.rename(columns = {'Prof':'all_profs'}, inplace=True)
df_prof_cannot_teach.rename(columns = {'Course':'all_courses'}, inplace=True)
# df_prof_cannot_teach

In [26]:
## for df of preferences for preps
df_preps_pref.rename(columns = {"Prof_name":"all_profs"}, inplace = True)
df_preps_pref = df_preps_pref.loc[df_preps_pref['all_profs'].isin(profs_with_enough_creds)]
# df_preps_pref

In [27]:
# multiplying the preps preferences by a scalar

colms = ['One','Two','Three','Four']

i = 0
for col in colms:
    df_preps_pref[col] = df_preps_pref[col]*pref_scaler
    i += 1
    
#df_preps_pref

In [28]:
all_profs = df_preferences["all_profs"]
#saving a variable with just the professor names
all_courses = df_goals["all_courses"]
#saving a variable with just the course names
all_times = df_times["all_times"].unique()
#saving a variable with just the different time assignments
all_days = df_times['Day'].unique()
#saving a variable with just the different days of the week

In [29]:
#all_courses

# Feasiblility

In [30]:
min_credits_profs_can_teach = 0 
max_credits_profs_can_teach = 0
min_goal_credits = 0
max_goal_credits = 0

for idx in range(df_credits_per_prof.shape[0]):
    row = df_credits_per_prof.iloc[idx]
    #finds the number of credits a professor can teach at minimum
    min_credits_profs_can_teach = min_credits_profs_can_teach + int(row['num_credits_min'])

#finds min sections or min total credits offered per class
for idx in range(df_goals.shape[0]):
    goal_row = df_goals.iloc[idx]
    #iterates through the credit goals for each class and saves the value for each class section in credit_per_class
    credit_per_class = df_credits.loc[df_credits['all_courses'] == goal_row['all_courses']]
        
    #uncomment the following to see the math that is used to find the minimum credits of a class that should be offered in a semester. 
    # the math should be the minimum preffered sections times the number of credits of the class. 
    # print(credit_per_class)
    # print(goal_row)
    
    for idx_2 in range(credit_per_class.shape[0]):
        credit_row = credit_per_class.iloc[idx_2]
        min_goal_credits = min_goal_credits + (int(goal_row['Goal_number_min']) * int(credit_row["number_of_credits"]))
    # print(credit_row.all_courses + " :\t" + str(min_goal_credits)+ " minimum credits") 
        #uncomment to see the total credits of a class

#finds every teachers maximum credit load saved as max_credits_prof_can_teach
for idx in range(df_credits_per_prof.shape[0]):
    row = df_credits_per_prof.iloc[idx]
    max_credits_profs_can_teach = max_credits_profs_can_teach + int(row['num_credits_max'])
# print(row) # uncomment to see a sample of the rows in df_credits per prof

#finds the maximum sections and credit count for each class and stores it in max_goal_credits
for idx in range(df_goals.shape[0]): 
    goal_row = df_goals.iloc[idx]
    credit_per_class = df_credits.loc[df_credits['all_courses'] == goal_row['all_courses']] #matches the course name found in the credit table with the course name found in the section offering goal table
    for idx_2 in range(credit_per_class.shape[0]):#loops through the table of classes and their standard credit count
        credit_row = credit_per_class.iloc[idx_2]
        max_goal_credits = max_goal_credits + int(goal_row['Goal_number_max'])*int(credit_row['number_of_credits']) # multiplies credits by maximum section offering count
#print(credit_row) # uncomment to see a sample of a row in credit_per_class
#print(goal_row) # uncomment to see a sample of a row in df_goals
        
#the following code checks the range for total available professors credits to the total needed credit offerings 
#print("is " + str(max_credits_profs_can_teach) +" < "+ str(min_goal_credits) + "? If, so, problem.") #uncomment to see comparisons
#print("is " + str(min_credits_profs_can_teach) +" > "+ str(max_goal_credits) + "? If, so, problem.") #uncomment to see second comparison

if max_credits_profs_can_teach < min_goal_credits:
        #print("infeasible")
        raise ValueError(f"Infeasible model: The minimum credits of classes desired is {min_goal_credits}; this is greater than the maximum the professors can teach ({max_credits_profs_can_teach}).")
elif min_credits_profs_can_teach > max_goal_credits:
        #print("infeasible")
        raise ValueError(f"Infeasible model: The maximum credits of classes desired is {max_goal_credits}; this is less than the minimum the professors can teach ({min_credits_profs_can_teach}).")
else:
    print("No obvious infeasibilities, proceed with model")
        
#should also add an error if the number of time options are not enough for the number of credits

No obvious infeasibilities, proceed with model


In [31]:
print(min_goal_credits)
print(max_credits_profs_can_teach)

24
24


In [32]:
mdl = Model(name="classes") #creates cplex model

In [33]:
# When there are 2 rows with the same "all_times" index, keep only one of them.
# This avoids complaints about duplicate variable names due to having
# 2 rows for each Math 347 meeting pattern.

df_times_unique_indices = df_times.drop_duplicates("all_times")

## New in July 2025:
### Just don't create assignment variables for Time_pref = 0 or Cannot_teach
* `df_time_groups_simple` already excludes classes with Goal_num_max = 0
* `all_profs` already excludes instructors with Credits_per_prof max = 0

In [34]:
df_time_groups_simple = df_time_groups.loc[df_time_groups['Class'].isin(all_courses)] 
# The rest of this code cell was commented out compared to Feb 2025.
#df_time_groups_simple
# df_time_group_with_time_index = df_time_groups_simple.merge(df_times_unique_indices, on=['Time_Group'])
# #df_time_group_with_time_index.loc[df_time_group_with_time_index['Class']=='Math 246']
# class_with_time_index_list = []

# for idx in range(df_time_group_with_time_index.shape[0]):
#     rows = df_time_group_with_time_index.iloc[idx]
#     class_with_time_index_list.append(rows['Class'] + "_" + str(rows['all_times']))
# df_time_group_with_time_index['class_with_time_index'] = class_with_time_index_list
# df_time_group_with_time_index

#assigned = mdl.binary_var_matrix(keys1=class_with_time_index_list, keys2=all_profs, name = 'time_assigned_%s_with_prof_%s')

# creating the model variables and adding the class time slots based on time groups to a df

Remove Cannot_teach from the set of combos for which we will create assignment variables.

In [35]:
df_prof_cannot_teach #642 rows
profs_and_courses = df_time_groups_simple.merge(all_profs, how='cross')

profs_and_courses.rename(columns = {'Class':'all_courses'}, inplace=True)
profs_and_courses #1416 rows
df_prof_cannot_teach['remove'] = True # now add a "remove" column to this, and join it to profs_and_courses.  Then drop rows with "remove" = TRUE
profs_and_courses = profs_and_courses.merge(df_prof_cannot_teach, how='left')
# The following df has more than 1416-642 rows, because df_prof_cannot_teach contains people with credits_max = 0 and profs_and_courses doesn't
profs_and_courses = profs_and_courses[profs_and_courses["remove"].isnull()] # 1044 rows
profs_and_courses.drop('remove', axis=1, inplace=True)
#profs_and_courses

Remove Time pref = 0 from the set of combos for which we will create assignment variables.

In [36]:
df_times_unique_indices #142 rows
prof_class_time = profs_and_courses.merge(df_times_unique_indices, on=['Time_Group'])
#prof_class_time #16,220 rows

In [37]:
#df_time_pref

In [38]:
df_time_pref_long = df_time_pref.melt(id_vars = ['all_profs'], var_name = 'all_times', value_name = 'Preference_value_of_time')
prof_class_time = prof_class_time.merge(df_time_pref_long, how='left') #16,220 rows
prof_class_time = prof_class_time.loc[prof_class_time['Preference_value_of_time'] > 0]
prof_class_time # 14,219 rows

,all_courses,Time_Group,all_profs,all_times,Day,Start_Time,End_Time,Notes,start_time_past_midnight,end_time_past_midnight,overlap_times,back_to_back_start_times,back_to_back_end_times,Preference_value_of_time
0,Math 347,L,ProfA,1,MWF,01:00:00,01:50:00,NaN,780,830,"[1, 4, 5]",[],[6],1
1,Math 347,L,ProfB,1,MWF,01:00:00,01:50:00,NaN,780,830,"[1, 4, 5]",[],[6],1
2,Math 246,H,ProfA,2,MWRF,08:00:00,08:50:00,NaN,480,530,[2],[],[3],1
3,Math 246,H,ProfA,3,MWRF,09:00:00,09:50:00,NaN,540,590,[3],[2],[],1
4,Math 246,H,ProfA,4,MWRF,01:00:00,01:50:00,NaN,780,830,"[1, 4]",[],[5],3
5,Math 246,H,ProfA,5,MWRF,02:00:00,02:50:00,NaN,840,890,"[1, 5]","[1, 4]",[6],1
6,Math 246,H,ProfA,6,MWRF,03:00:00,03:50:00,NaN,900,950,[6],"[1, 5]",[],3
7,Math 246,H,ProfB,2,MWRF,08:00:00,08:50:00,NaN,480,530,[2],[],[3],5
8,Math 246,H,ProfB,3,MWRF,09:00:00,09:50:00,NaN,540,590,[3],[2],[],5
9,Math 246,H,ProfB,4,MWRF,01:00:00,01:50:00,NaN,780,830,"[1, 4]",[],[5],3


Name the assignment variables, then create them.

In [39]:
#prof_class_time['combo_name'] = prof_class_time['all_courses'] + '_' + str(prof_class_time['all_times'])
prof_class_time = prof_class_time.assign(combo_name = prof_class_time.all_courses + '_' + prof_class_time.all_times.astype(str) + '_with_prof_' + prof_class_time.all_profs)
# Need all_courses_and_times later on to unique-ify Math 347
prof_class_time = prof_class_time.assign(all_courses_and_times = prof_class_time.all_courses + '_' + prof_class_time.all_times.astype(str))

#prof_class_time

In [40]:
#prof_class_time.loc[prof_class_time.all_profs=="MohammadAziz"]

In [41]:
assigned = mdl.binary_var_list(prof_class_time.combo_name)
#assigned

### To do: investigate whether keeping the time info in df_assigned lets us replace df_time_assigned with df_assigned, later on.

In [42]:
#creating the assignment dataframe for with the variables
df_assigned = prof_class_time
df_assigned['assigned'] = assigned
df_assigned.drop(['combo_name','Notes'], axis=1, inplace=True)  # could also drop everything else except assigned, all_courses, all_times, and all_profs
# but maybe keeping it all together will reduce the number of dataframes we need to create later?
df_assigned
df_assigned.drop(['Time_Group','Day','Start_Time','End_Time','start_time_past_midnight','end_time_past_midnight'], axis=1, inplace=True)
df_assigned.drop(['overlap_times', 'back_to_back_start_times','back_to_back_end_times','Preference_value_of_time'], axis=1, inplace=True)

In [43]:
# df_assigned = DataFrame({'assigned': assigned})
# df_assigned.index.names = ['all_courses_and_times','all_profs']
# df_assigned = df_assigned.reset_index()
# time_course_spliting = df_assigned["all_courses_and_times"].str.split("_", n=1, expand=True)
# df_assigned['all_courses'] = time_course_spliting[0]
# df_assigned['all_times'] = time_course_spliting[1]
# df_assigned['all_times'] = df_assigned['all_times'].astype(int)
# df_assigned
# #creating the assignment dataframe for with the variables

In [44]:
#df_assigned[df_assigned["all_courses"] == "Math 10"]
#df_time_groups_simple # 10 and 20 are here
#df_times_unique_indices

this is making the back to back columns be lists of integers and not strings

In [45]:
# df_time_assigned_cleaned = df_times.drop(columns = ['Notes'])
# df_time_assigned_cleaned = df_time_assigned_cleaned.dropna()
# #dropping na time values
# dict_back_to_back_s = {}
# dict_back_to_back_e = {}
# dict_overlaps = {}
# for time in df_time_assigned_cleaned['all_times'].unique():
#     #print(time)
#     df_by_time = df_times.loc[df_times['all_times']==time]
#     #print(df_by_time)
#     back_back_s = df_by_time['back_to_back_start_times'].unique()[0]
#     back_back_e = df_by_time['back_to_back_end_times'].unique()[0]
#     overlaps = df_by_time['overlap_times'].unique()[0]
#     back_to_back_list_s = re.findall("[0-9]+", back_back_s)
#     back_to_back_list_e = re.findall("[0-9]+", back_back_e)
#     overlap_list = re.findall("[0-9]+", overlaps)
#     #back_back_list = back_back.split("[")[-1].split("]")
#     back_to_back_list_s = [int(times) for times in back_to_back_list_s]
#     back_to_back_list_e = [int(times) for times in back_to_back_list_e]
#     overlap_list = [int(times) for times in overlap_list]
#     #prof, at time, and class, assignment variables
#     dict_back_to_back_s[time] = back_to_back_list_s
#     dict_back_to_back_e[time] = back_to_back_list_e
#     dict_overlaps[time] = overlap_list
# df_time_assigned_cleaned['back_to_back_start_times'] = df_time_assigned_cleaned['all_times'].map(dict_back_to_back_s)
# df_time_assigned_cleaned['back_to_back_end_times'] = df_time_assigned_cleaned['all_times'].map(dict_back_to_back_e)
# df_time_assigned_cleaned['overlap_times'] = df_time_assigned_cleaned['all_times'].map(dict_overlaps)
# df_times = df_time_assigned_cleaned
# # df_time_assigned_cleaned.drop(columns=['back_to_back_times'])
# # creating a column that has the back to back times as a list and not a string

Brynn's new code to fix problem with asynchronous classes, 10 Feb 2025

In [46]:
df_time_assigned_cleaned = df_times.drop(columns = ['Notes'])
#print(df_time_assigned_cleaned)
### getting asynch classes so they aren't dropped when na are dropped
df_add_na = df_time_assigned_cleaned.loc[df_time_assigned_cleaned['Day'].isna()]
#print(df_add_na)
df_time_assigned_cleaned = df_time_assigned_cleaned.dropna()
#print(df_time_assigned_cleaned)
#dropping na time values
dict_back_to_back_s = {}
dict_back_to_back_e = {}
dict_overlaps = {}
for time in df_time_assigned_cleaned['all_times'].unique():
    #print(time)
    df_by_time = df_times.loc[df_times['all_times']==time]
    #print(df_by_time)
    back_back_s = df_by_time['back_to_back_start_times'].unique()[0]
    back_back_e = df_by_time['back_to_back_end_times'].unique()[0]
    overlaps = df_by_time['overlap_times'].unique()[0]
    back_to_back_list_s = re.findall("[0-9]+", back_back_s)
    back_to_back_list_e = re.findall("[0-9]+", back_back_e)
    overlap_list = re.findall("[0-9]+", overlaps)
    #back_back_list = back_back.split("[")[-1].split("]")
    back_to_back_list_s = [int(times) for times in back_to_back_list_s]
    back_to_back_list_e = [int(times) for times in back_to_back_list_e]
    overlap_list = [int(times) for times in overlap_list]
    #prof, at time, and class, assignment variables
    dict_back_to_back_s[time] = back_to_back_list_s
    dict_back_to_back_e[time] = back_to_back_list_e
    dict_overlaps[time] = overlap_list
df_time_assigned_cleaned['back_to_back_start_times'] = df_time_assigned_cleaned['all_times'].map(dict_back_to_back_s)
df_time_assigned_cleaned['back_to_back_end_times'] = df_time_assigned_cleaned['all_times'].map(dict_back_to_back_e)
df_time_assigned_cleaned['overlap_times'] = df_time_assigned_cleaned['all_times'].map(dict_overlaps)
#### adding the asych classes back
df_times = pd.concat([df_time_assigned_cleaned,df_add_na])
# df_time_assigned_cleaned.drop(columns=['back_to_back_times'])
# creating a column that has the back to back times as a list and not a string
#df_times


In [47]:
mdl.print_information()

Model: classes
 - number of variables: 12
   - binary=12, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP


# Constraints for previous schedules

In [48]:
#setting the variables from previous solution to have to be a part of the new solution
# TPS added priority flags 11/19/24 
# I'm honestly not sure if they work or not because I don't have a previous schedule to test the code with, but they seem fine with the other if/then constraints

if prev_schedule:
    partial_results_file = partial_results_file.merge(df_assigned[['all_times','all_courses', 'all_courses_and_times']].drop_duplicates(), on= ['all_times','all_courses'])
    partial_results_file.drop_duplicates([["all_courses_and_times", "all_profs"]], inplace=True)
    for idx in range(partial_results_file.shape[0]):
        par_res_row = partial_results_file.iloc[idx].copy()
#         if not (par_res_row['all_courses'] == "Math 347" and (par_res_row["Day"] == "T" or par_res_row['Day']=="R")):
        if pd.notna(par_res_row['all_times']) and pd.notna(par_res_row['all_courses']) and pd.notna(par_res_row['all_profs']):
            var = df_assigned.loc[df_assigned['all_times']==par_res_row['all_times']].loc[df_assigned['all_courses']==par_res_row['all_courses']].loc[df_assigned['all_profs']==par_res_row['all_profs']].reset_index()
            mdl.add_constraint(mdl.sum(var['assigned'])==len(var['assigned'])).priority = Priority.MEDIUM
        elif pd.notna(par_res_row['all_times']) and pd.notna(par_res_row['all_profs']):
            var = df_assigned.loc[df_assigned['all_times']==par_res_row['all_times']].loc[df_assigned['all_profs']==par_res_row['all_profs']].reset_index()
            prev_assign = partial_results_file.loc[partial_results_file['all_times']==par_res_row['all_times']].loc[partial_results_file['all_profs']==par_res_row['all_profs']]
            mdl.add_constraint(mdl.sum(var['assigned'])>=prev_assign.shape[0]).priority = Priority.MEDIUM
        elif pd.notna(par_res_row['all_courses']) and pd.notna(par_res_row['all_profs']):
            var = df_assigned.loc[df_assigned['all_courses']==par_res_row['all_courses']].loc[df_assigned['all_profs']==par_res_row['all_profs']].reset_index()
            prev_assign = partial_results_file.loc[partial_results_file['all_courses']==par_res_row['all_courses']].loc[partial_results_file['all_profs']==par_res_row['all_profs']]
            mdl.add_constraint(mdl.sum(var['assigned'])>=prev_assign.shape[0]).priority = Priority.MEDIUM
        elif pd.notna(par_res_row['all_courses']) and pd.notna(par_res_row['all_times']):
            var = df_assigned.loc[df_assigned['all_courses']==par_res_row['all_courses']].loc[df_assigned['all_times']==par_res_row['all_times']].reset_index()
            prev_assign = partial_results_file.loc[partial_results_file['all_courses']==par_res_row['all_courses']].loc[partial_results_file['all_times']==par_res_row['all_times']]
            mdl.add_constraint(mdl.sum(var['assigned'])>=prev_assign.shape[0]).priority = Priority.MEDIUM

In [49]:
mdl.print_information() # When prev_schedule == False, this does not add any constraints, good

Model: classes
 - number of variables: 12
   - binary=12, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP


### Constraint:  No overlapping classes

In [50]:
df_time_assigned = df_times.loc[df_times['all_times'].isin(df_assigned['all_times'])].copy()
df_time_assigned.drop(columns = ['Day','Start_Time','End_Time','Time_Group'], inplace=True)
df_time_assigned = df_assigned.merge(df_times, on='all_times')
#df_time_assigned

In [51]:
# # TPS added inplace priority markers, 11/19/24
# for times in df_time_assigned['all_times'].unique():
#     df_time_seperate = df_time_assigned.loc[df_time_assigned['all_times']==times].copy().reset_index() # fixed the "value being set on a copy" warning
#     df_time_seperate = df_time_seperate.dropna()
#     df_time_overlap_options = df_time_assigned.loc[df_time_assigned['all_times'].isin(df_time_seperate['overlap_times'][0])]
#     for prof in all_profs:
#         #print(f'{prof}, time {times}')
#         df_overlap_prof = df_time_overlap_options.loc[df_time_overlap_options['all_profs']==prof]
#         # Drop duplicate course-time pairs (avoid problems with Math 347)
#         df_overlap_prof = df_overlap_prof.drop_duplicates(["all_courses_and_times"]) # Added by Dr. Abra on 8/1/2024

#         # The number of classes `prof` can teach in timeslot `times` is <= 1.
#         mdl.add_constraint(mdl.sum(df_overlap_prof.loc[df_overlap_prof['all_times']==times]['assigned'])<=1).priority = Priority.VERY_HIGH
#         if len(df_overlap_prof['all_times'].unique()) >1:
#             #print(len(df_overlap_prof['all_times'].unique()))
#             # If there are other timeslots that overlap with this one, and `prof` teaches at timeslot `times`,
#             # then they cannot teach during any of the *other* overlapping timeslots
#             #if times == 55:
#                 #print(sum(df_overlap_prof.loc[df_overlap_prof['all_times']==times]['assigned']))
#             mdl.add_if_then(mdl.sum(df_overlap_prof.loc[df_overlap_prof['all_times']==times]['assigned'])==1, mdl.sum(df_overlap_prof.loc[df_overlap_prof['all_times']!=times]['assigned'])==0).priority = Priority.VERY_HIGH
# #df_overlap_prof
# #adding the constraint for overlapping times, look at the list of overlapping times, find the prof and all those times say if one not the other

Newly factored out in Nov 2025

In [52]:
constraint_no_overlapping_classes(df_time_assigned, all_profs, mdl)

docplex.mp.Model['classes']

In [53]:
mdl.print_information()

Model: classes
 - number of variables: 18
   - binary=18, integer=0, continuous=0
 - number of constraints: 24
   - linear=12, indicator=6, equiv=6
 - parameters: defaults
 - objective: none
 - problem type is: MILP


### course course time constraints

In [54]:
# # TPS added priority flags, 11/19/2024
# df_2_course_time_avoid_constraint = df_course_course_time_constraints.loc[df_course_course_time_constraints['Type']=="Avoid"].reset_index()
# for idx in range(df_2_course_time_avoid_constraint.shape[0]):
#     course_pairs = df_2_course_time_avoid_constraint.iloc[idx]
#     df_assigned_course_1 = df_time_assigned.loc[df_time_assigned['all_courses']==course_pairs['Course_1']]
#     df_assigned_course_2 = df_time_assigned.loc[df_time_assigned['all_courses']==course_pairs['Course_2']]
#     if df_assigned_course_1.shape[0] >0  and df_assigned_course_2.shape[0]>0:
#         for time in df_assigned_course_1['all_times'].unique():
#             df_1_by_time = df_assigned_course_1.loc[df_assigned_course_1['all_times'] == time].reset_index()
#             df_overlaps = df_assigned_course_2.loc[df_assigned_course_2['all_times'].isin(df_1_by_time['overlap_times'][0])]
#             if df_overlaps.shape[0] > 0:
#                 mdl.add_if_then(mdl.sum(df_1_by_time['assigned'])>=1, mdl.sum(df_overlaps['assigned']) == 0).priority = Priority.MEDIUM
#                 mdl.add_if_then(mdl.sum(df_overlaps['assigned'])>=1, mdl.sum(df_1_by_time['assigned']) == 0).priority = Priority.MEDIUM

Newly factored-out in Nov 2025

In [55]:
### Block one: course_course_time_constraints AVOID

constraint_course_course_time_avoid (df_course_course_time_constraints, df_time_assigned, mdl)

docplex.mp.Model['classes']

In [56]:
df_course_course_time_constraints
mdl.print_information()

Model: classes
 - number of variables: 18
   - binary=18, integer=0, continuous=0
 - number of constraints: 24
   - linear=12, indicator=6, equiv=6
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [57]:
# df_2_course_time_require_constraint = df_course_course_time_constraints.loc[df_course_course_time_constraints['Type']=="Require"].reset_index()
# for idx in range(df_2_course_time_require_constraint.shape[0]):
#     course_pair = df_2_course_time_require_constraint.iloc[idx]
#     df_course_1 = df_time_assigned.loc[df_time_assigned['all_courses'] == course_pair['Course_1']]
#     df_course_2 = df_time_assigned.loc[df_time_assigned['all_courses'] == course_pair['Course_2']]
#     list_of_overlapped_times_accounted_for = []
#     if df_course_1.shape[0] >0 and df_course_2.shape[0] >0:
#         for times in df_course_1['all_times'].unique():
#             df_time_1 = df_course_1.loc[df_course_1['all_times']==times].reset_index()
#             if df_time_1.shape[0]>0:
#                 if times not in list_of_overlapped_times_accounted_for:
#                     list_of_overlapped_times_accounted_for += df_time_1['overlap_times'].iloc[0]
#                     df_time_2 = df_course_2.loc[df_course_2['all_times'].isin(df_time_1['overlap_times'].iloc[0])] # change .isin(df_time_1['overlap_times']) to == times, if same timeslot required
#                     if df_time_2.shape[0]>0:
#                         mdl.add_if_then(mdl.sum(df_time_1['assigned']) >= 1, mdl.sum(df_time_2['assigned'])>=1).priority = Priority.MEDIUM
#                         mdl.add_if_then(mdl.sum(df_time_2['assigned'])>=1,  mdl.sum(df_time_1['assigned'])>=1).priority = Priority.MEDIUM # this might be redundant 

Newly factored-out in Nov 2025

In [58]:
### Block 2: course_course_time_constraints REQUIRE

constraint_course_course_time_require (df_course_course_time_constraints, df_time_assigned, mdl)

docplex.mp.Model['classes']

In [59]:
mdl.print_information() # Currently no REQUIREs, so this doesn't add any variables or constraints

Model: classes
 - number of variables: 18
   - binary=18, integer=0, continuous=0
 - number of constraints: 24
   - linear=12, indicator=6, equiv=6
 - parameters: defaults
 - objective: none
 - problem type is: MILP


## Change from Fall_2025_schedule.ipynb:  
### Commented out this cell because we are moving course_course_time_prefs into low-priority constraints.

In [60]:
# course_course_time_pref_series = []
# # This was commented out in preps-and-some-b2bs-v3.  why?
# for idx in range(df_course_course_time_prefs.shape[0]):
#     course_pair = df_course_course_time_prefs.iloc[idx]
#     df_course_1 = df_time_assigned.loc[df_time_assigned['all_courses'] == course_pair['Course_1']]
#     df_course_2 = df_time_assigned.loc[df_time_assigned['all_courses'] == course_pair['Course_2']]
#     if df_course_1.shape[0]>0 and df_course_2.shape[0]>0:
#         for times in df_course_1['all_times'].unique():
#             df_time_1 = df_course_1.loc[df_course_1['all_times']==times].reset_index()
#             df_time_2 = df_course_2.loc[df_course_2['all_times'].isin(df_time_1['overlap_times'].iloc[0])]
#             if df_time_1.shape[0] > 0 and df_time_2.shape[0]>0:
#                 course_course_time_pref_series.append(mdl.max(df_time_1['assigned'])*mdl.max(df_time_2['assigned'])*course_pair['Pref'])

In [61]:
df_course_time_prefs_with_assign = df_course_time_prefs.merge(df_assigned, left_on=['Course', 'Time'], right_on=['all_courses', 'all_times'])
course_time_pref_series = df_course_time_prefs_with_assign['assigned']*df_course_time_prefs_with_assign['Pref']
course_time_pref_series

Series([], dtype: object)

In [62]:
# Newly factored-out in Nov 2025
# Constraints about what time indices can and can't happen for specific classes
constraint_course_time(df_course_time_constraints,df_assigned,mdl)

docplex.mp.Model['classes']

In [63]:
mdl.print_information()

Model: classes
 - number of variables: 18
   - binary=18, integer=0, continuous=0
 - number of constraints: 24
   - linear=12, indicator=6, equiv=6
 - parameters: defaults
 - objective: none
 - problem type is: MILP


# Distribution Throughout day
Courses with max sections less than or equal to `max_number_sections_to_be_at_diff_times`, cannot happen at overlapping times for any professor

In [64]:
# df_time_assigned = df_times.loc[df_times['all_times'].isin(df_assigned['all_times'])].copy()
# df_time_assigned.drop(columns = ['Day','Start_Time','End_Time','Time_Group'], inplace=True)
# df_time_assigned = df_assigned.merge(df_times, on='all_times')
# for times in df_time_assigned['all_times'].unique():
#     df_time_seperate = df_time_assigned.loc[df_time_assigned['all_times']==times].copy().reset_index() # fixed the "value being set on a copy" warning
#     df_time_seperate = df_time_seperate.dropna()
#     df_time_overlap_options = df_time_assigned.loc[df_time_assigned['all_times'].isin(df_time_seperate['overlap_times'][0])]
#     for course in all_courses:
#         max_sections = df_goals.loc[df_goals['all_courses'] == course]['Goal_number_max'].unique()[0]
#         if max_sections <= max_number_sections_to_be_at_diff_times:
#             df_overlap_course = df_time_overlap_options.loc[df_time_overlap_options['all_courses']==course]
#             # Drop duplicate course-time pairs (avoid problems with Math 347)
            
#             df_overlap_course = df_overlap_course.drop_duplicates(["all_courses_and_times", 'all_profs']) 
#             # The number of classes happening in timeslot `times` is <= 1.
#             mdl.add_constraint(mdl.sum(df_overlap_course.loc[df_overlap_course['all_times']==times]['assigned'])<=1).priority = Priority.MEDIUM
#             if len(df_overlap_course['all_times'].unique()) >1:
#                 #print(len(df_overlap_prof['all_times'].unique()))
#                 # If there are other timeslots that overlap with this one, and `prof` teaches at timeslot `times`,
#                 # then they cannot teach during any of the *other* overlapping timeslots
#                 #if times == 55:
#                     #print(sum(df_overlap_prof.loc[df_overlap_prof['all_times']==times]['assigned']))
#                 mdl.add_if_then(mdl.sum(df_overlap_course.loc[df_overlap_course['all_times']==times]['assigned'])==1, mdl.sum(df_overlap_course.loc[df_overlap_course['all_times']!=times]['assigned'])==0).priority = Priority.MEDIUM
# #df_overlap_prof

Brynn 10 Feb 2025

In [65]:
# df_time_assigned = df_times.loc[df_times['all_times'].isin(df_assigned['all_times'])].copy()
# df_time_assigned.drop(columns = ['Day','Start_Time','End_Time','Time_Group'], inplace=True)
# df_time_assigned = df_assigned.merge(df_times, on='all_times')
# for times in df_time_assigned['all_times'].unique():
#     df_time_seperate = df_time_assigned.loc[df_time_assigned['all_times']==times].copy().reset_index() # fixed the "value being set on a copy" warning
#     df_time_seperate = df_time_seperate.dropna()
#     ##### added for asych class purposes
#     if not df_time_seperate.empty:
#         df_time_overlap_options = df_time_assigned.loc[df_time_assigned['all_times'].isin(df_time_seperate['overlap_times'][0])]
#         for course in all_courses:
#             max_sections = df_goals.loc[df_goals['all_courses'] == course]['Goal_number_max'].unique()[0]
#             if max_sections <= max_number_sections_to_be_at_diff_times:
#                 df_overlap_course = df_time_overlap_options.loc[df_time_overlap_options['all_courses']==course]
#                 # Drop duplicate course-time pairs (avoid problems with Math 347)
            
#                 df_overlap_course = df_overlap_course.drop_duplicates(["all_courses_and_times", 'all_profs']) 
#                 # The number of classes happening in timeslot `times` is <= 1.
#                 mdl.add_constraint(mdl.sum(df_overlap_course.loc[df_overlap_course['all_times']==times]['assigned'])<=1).priority = Priority.MEDIUM
#                 if len(df_overlap_course['all_times'].unique()) >1:
#                     #print(len(df_overlap_prof['all_times'].unique()))
#                     # If there are other timeslots that overlap with this one, and `prof` teaches at timeslot `times`,
#                     # then they cannot teach during any of the *other* overlapping timeslots
#                     #if times == 55:
#                         #print(sum(df_overlap_prof.loc[df_overlap_prof['all_times']==times]['assigned']))
#                     mdl.add_if_then(mdl.sum(df_overlap_course.loc[df_overlap_course['all_times']==times]['assigned'])==1, mdl.sum(df_overlap_course.loc[df_overlap_course['all_times']!=times]['assigned'])==0).priority = Priority.MEDIUM
# #df_overlap_prof



Brynn 14 Feb 2025

In [66]:
# # going through all possible times
# for times in df_time_assigned['all_times'].unique():
#     # making dataframe with only those times 
#     df_time_seperate = df_time_assigned.loc[df_time_assigned['all_times']==times].copy().reset_index()
#     df_time_seperate = df_time_seperate.dropna()
#     #print(df_time_seperate)
#     # for asych classes to work
#     if not df_time_seperate['overlap_times'].empty:
#         #print(df_time_seperate['overlap_times'][0])
        
#         # getting all of the times that would overlap with the specific time frame
#         df_time_overlap_options = df_time_assigned.loc[df_time_assigned['all_times'].isin(df_time_seperate['overlap_times'][0])]
#         #print(df_time_seperate['overlap_times'][0])
#         # going through all the courses 
#         for course in all_courses:
#             #print(df_goals.loc[df_goals['all_courses'] == course]['Goal_number_max'].unique()[0])
#             if df_goals.loc[df_goals['all_courses'] == course]['Goal_number_max'].unique()[0] <= max_number_sections_to_be_at_diff_times:
#                 # getting dataframe with only that specific course
#                 df_time_assigned_course_diff = df_time_assigned.loc[df_time_assigned['all_courses'] == course]
#                 # only getting overlap times for the specific course
#                 df_overlap_course = df_time_overlap_options.loc[df_time_overlap_options['all_courses'] == course]
#                 # dropping duplicates
#                 df_overlap_course = df_overlap_course.drop_duplicates(["all_courses_and_times"])   
#                 #print(df_overlap_course['all_times'])
#                 # adding constraint that the course can not be taught
#                 #print(df_overlap_course)
#                 # if there are any other timeslots that overlap with this one and course is taught during this one
#                 # then the course cannot be offered during any of the other overlapping timeslots
#                 # Priority changed from Very High to Medium by Dr. Abra after Brynn tested her code
#                 mdl.add_constraint(mdl.sum(df_overlap_course.loc[df_overlap_course['all_times']==times]['assigned'])<=1).priority = Priority.MEDIUM #VERY_HIGH
#                 if len(df_overlap_course['all_times'].unique()) >1:
#                     mdl.add_if_then(mdl.sum(df_overlap_course.loc[df_overlap_course['all_times']==times]['assigned'])<=max_number_sections_to_be_at_diff_times, mdl.sum(df_overlap_course.loc[df_overlap_course['all_times']!=times]['assigned'])==0).priority = Priority.MEDIUM #VERY_HIGH

In [67]:
mdl.print_information()

Model: classes
 - number of variables: 18
   - binary=18, integer=0, continuous=0
 - number of constraints: 24
   - linear=12, indicator=6, equiv=6
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [68]:
constraint_dist_throughout_day(df_times,df_assigned,df_goals,max_number_sections_to_be_at_diff_times,mdl) # double-check:  is this adding a different number of constraints than the .ipynb version of the code?

docplex.mp.Model['classes']

In [69]:
mdl.print_information()

Model: classes
 - number of variables: 18
   - binary=18, integer=0, continuous=0
 - number of constraints: 30
   - linear=18, indicator=6, equiv=6
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [70]:
df_times_with_length = df_times#.merge(num_hours_per_meeting_time_df, on=['all_times'])
#print(df_times_with_length)
#df_assigned_with_week_full_indices = df_assignments_across_week.merge(df_times, on=['index_across_week'])
#print(df_assigned_with_week_full_indices)
df_assigned_with_week_indices_and_meet_times = df_assigned.merge(df_times_with_length, on=['all_times'])
# print(df_assigned_with_week_indices_and_meet_times)
df_week_simplified_time_and_assignment = df_assigned_with_week_indices_and_meet_times.drop(columns=['Start_Time', 'End_Time', 'Day'])
df_week_simplified_assignment_and_credits = df_week_simplified_time_and_assignment.merge(df_credits, on=['all_courses'])
df_week_individual_simple = df_week_simplified_assignment_and_credits.drop_duplicates(subset = ['all_profs', 'all_courses','all_times'])
# df_week_individual_simple

In [71]:
mdl.print_information() # no change, good

Model: classes
 - number of variables: 18
   - binary=18, integer=0, continuous=0
 - number of constraints: 30
   - linear=18, indicator=6, equiv=6
 - parameters: defaults
 - objective: none
 - problem type is: MILP


### Constrain each professor to teach the correct number of credits

In [72]:
# # Skip this while debugging
# # summing all a profs assigment variables and saying that it has to be less than and greater than the min and max number of credits per that prof
# for prof, course_assignments in df_week_individual_simple.groupby(by='all_profs'):
#     #print(prof)
#     mdl.add_constraint(mdl.sum(course_assignments['assigned']*course_assignments['number_of_credits']) <= (df_credits_per_prof.loc[lambda df:df['all_profs'] == prof, 'num_credits_max'].values[0]), ctname = f"max credits per {prof}").priority = Priority.MEDIUM 
#     mdl.add_constraint(mdl.sum(course_assignments['assigned']*course_assignments['number_of_credits']) >= (df_credits_per_prof.loc[lambda df:df['all_profs'] == prof, 'num_credits_min'].values[0]), ctname = f"min credits per {prof}").priority = Priority.MEDIUM

In [73]:
mdl.print_information() # The number of constraints should now have increased by 2*the number of instructors with Goal_num_max > 0

Model: classes
 - number of variables: 18
   - binary=18, integer=0, continuous=0
 - number of constraints: 30
   - linear=18, indicator=6, equiv=6
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [74]:
constraint_prof_num_credits(df_assigned, df_credits_per_prof, df_credits, mdl) # should modify mdl in place

docplex.mp.Model['classes']

In [75]:
mdl.print_information() # The number of constraints should now have increased by 2*the number of instructors with Goal_num_max > 0

Model: classes
 - number of variables: 18
   - binary=18, integer=0, continuous=0
 - number of constraints: 34
   - linear=22, indicator=6, equiv=6
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [76]:
# If the code cell above produces an error, it might be because someone's name is spelled differently
# on the Credits_per_prof sheet than on the Preferences sheet.
# This is the person whose name is spelled differently:
#print(prof)
#print(df_credits_per_prof.loc[lambda df:df['prof_name'] == prof, 'num_credits_max']) # if this is an empty Series, that explains the "positional indexer" error.

## Constraining each class to be taught the correct number of times

If you get "Error: Adding trivially infeasible linear constraint: '0 >= 1'", double-check that all courses that should be in Meeting_Patterns_large.xlsx, are there.  Don't think we need to re-run Overlapping_times_version_1.ipynb unless the class has a new "type" of meeting pattern.

Another possible cause of this error is if two rows in Meeting_Patterns have the same Time_Index when they shouldn't (or, more specifically, if all meeting patterns for a class' Time_Group have the same Time_Index as the rows for a different Time_Group).

In [77]:
# #Skip this while debugging
# #summing all of a class assigment variables and saying that it has to be less than and greater than the min and max number of sections per that class
# for class_name in all_courses:
#     #print(class_name)
#     #print(df_goals.loc[lambda df: df["all_courses"] == class_name, "Goal_number_min"].iloc[0])
#     mdl.add_constraint(mdl.sum(df_assigned.loc[lambda df: df["all_courses"] == class_name, "assigned"]) <= (df_goals.loc[lambda df: df["all_courses"] == class_name, "Goal_number_max"].iloc[0]), ctname = f"max number of {class_name} sections").priority = Priority.MEDIUM
#     mdl.add_constraint(mdl.sum(df_assigned.loc[lambda df: df["all_courses"] == class_name, "assigned"]) >= (df_goals.loc[lambda df: df["all_courses"] == class_name, "Goal_number_min"].iloc[0]), ctname = f"min number of {class_name} sections").priority = Priority.MEDIUM

In [78]:
mdl.print_information()

Model: classes
 - number of variables: 18
   - binary=18, integer=0, continuous=0
 - number of constraints: 34
   - linear=22, indicator=6, equiv=6
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [79]:
constraint_each_class_taught_correct_number_of_times(all_courses,df_assigned,df_goals,mdl)

docplex.mp.Model['classes']

In [80]:
mdl.print_information()

Model: classes
 - number of variables: 18
   - binary=18, integer=0, continuous=0
 - number of constraints: 38
   - linear=26, indicator=6, equiv=6
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [81]:
#class_name = "Math 10"
#df_goals.loc[lambda df: df["all_courses"] == class_name, "Goal_number_min"].iloc[0]
#df_assigned.loc[lambda df: df["all_courses"] == "Math 114", "assigned"]
#df_assigned[df_assigned["all_courses"] == "Math 10"]

## Constraint:  No one teaches more than 1 section of Math 10+20
Not using this for Fall 2025, to test out a model where Katrina teaches multiple lectures of Math 10/20.

In [82]:
# Take only the rows that correspond to Math 10/20
# row_10_20 = (df_week_individual_simple["all_courses"] == "Math 10") | (df_week_individual_simple["all_courses"] == "Math 20")
# for prof, course_assignments in df_week_individual_simple[row_10_20].groupby(by='all_profs'):
#     mdl.add_constraint(mdl.sum(course_assignments['assigned']) <= 1, ctname = f"10/20 limit for {prof}").priority = Priority.MEDIUM

## Constraint: Course-Course Prof Constraints
 - TPS 11/19/24
some courses can't be/have to be taught by the same prof

Avoid = If a prof teaches a section of course_1, they cannot teach any of course_2, and vice versa

Require = If a prof teaches x sections of course_1, they must teach exactly x sections of course_2, and vice versa. This assumes that the same number of sections of course_1 and course_2 are running

You (Theo) should make sure to test if this still works when there are more than one section of course_1 and course_2

In [83]:
# Newly factored out in Nov 2025
if use_course_course_prof:
    constraint_course_course_prof_avoid_and_require(df_course_course_prof_constraints, df_assigned, df_goals, all_profs, mdl)

In [84]:
mdl.print_information()

Model: classes
 - number of variables: 18
   - binary=18, integer=0, continuous=0
 - number of constraints: 38
   - linear=26, indicator=6, equiv=6
 - parameters: defaults
 - objective: none
 - problem type is: MILP


## Constraint: Profs can't teach certain courses
### Commented out in July 2025
- TPS 11/19/24

In [85]:
# df_prof_cannot_teach_assigned = df_prof_cannot_teach.merge(df_assigned)
# df_prof_cannot_teach_assigned

# for index, row in df_prof_cannot_teach_assigned.iterrows():
#     #print(row['assigned'])
#     prof = row['all_profs']
#     clas = row['all_courses']
#     mdl.add_constraint(row['assigned'] == 0, ctname = f'{prof} cannot teach {clas}').priority = Priority.HIGH

## Constraint: Time preferences of 0 are impossible
### Commented out in July 2025
- TPS 11/19/24

In [86]:
# df_week_individual_simple
# df_time_pref_long = df_time_pref.melt(id_vars = ["all_profs"], var_name = "all_times", value_name = "Preference_value_of_time")
# df_time_pref_long = df_time_pref_long.rename(columns = {"Prof_name":"all_profs"})
# df_time_pref_long
# df_time_pref_0 = df_time_pref_long.loc[df_time_pref_long['Preference_value_of_time']==0] 
# df_time_pref_0
# #type(df_time_pref_long)

# df_assigned
# df_time_pref_0_assigned = df_time_pref_0.merge(df_assigned)
# df_time_pref_0_assigned # Why don't Math 347 and Math 246 both have assignment variables for the time preference of time slot 1? Because Math 347 is L and Math 246 is H
# #They aren't supposed to have two classes for these time slots because the time slots in question have different meeting schemes

# for index, row in df_time_pref_0_assigned.iterrows():
#     prof = row['all_profs']
#     t = row['all_times']
#     #print(row)
#     #This stuff all adds a constraint.......................................................... ...v
#     #                                                                                                And this stuff sets priority inplace
#     #Which seems confusing because stuff with an = after it shouldn't work like that but it's fine I guess?
#     mdl.add_constraint(row['assigned'] == 0, ctname = f'{prof} has time preference of 0 at time {t}').priority = Priority.MEDIUM

# Preferences about courses
this is adding class pref per prof and time pref per prof to the satisfaction series.

In [87]:
df_class_pref_long = df_preferences.melt(id_vars = ["all_profs"], var_name = "all_courses", value_name = "Preference_value")
df_class_pref_long = df_class_pref_long.rename(columns = {"prof_name":"all_profs"})
df_time_pref_long = df_time_pref.melt(id_vars = ["all_profs"], var_name = "all_times", value_name = "Preference_value_of_time")
df_time_pref_long = df_time_pref_long.rename(columns = {"Prof_name":"all_profs"})
df_assigned_with_prefs = df_assigned.merge(df_class_pref_long, on = ["all_profs", "all_courses"])
df_assigned_with_prefs = df_assigned_with_prefs.merge(df_time_pref_long, on = ['all_profs', 'all_times'])
class_prefs_satisfaction_series = df_assigned_with_prefs["assigned"] * df_assigned_with_prefs['Preference_value']
time_prefs_satisfaction_series = df_assigned_with_prefs["assigned"] * df_assigned_with_prefs['Preference_value_of_time']
# df_assigned_with_prefs
# summing the preferences across prof class and times

In [88]:
mdl.print_information() # no change, good

Model: classes
 - number of variables: 18
   - binary=18, integer=0, continuous=0
 - number of constraints: 38
   - linear=26, indicator=6, equiv=6
 - parameters: defaults
 - objective: none
 - problem type is: MILP


# Number of Preps per Professor (preferences)

### Small change from Feb_2025:
Moved this bit above the Preferences about back-to-back courses so we can try to avoid making an entire copy of df_time_assigned.

In [89]:
#creating binary variable matrix where 0 means professor teaches no sections of class and 
#1 means professor teaches at least 1 section
class_assigned = mdl.binary_var_matrix(keys1 = all_profs, keys2 = all_courses, name = "%s_at_%s")
df_class_assigned = DataFrame({'class_assigned': class_assigned})
df_class_assigned.index.names = ['all_profs', 'all_courses']
df_class_assigned = df_class_assigned.reset_index()

look at assigned variable in df_time_assigned_clean, find one professor with one specific class and make sub-dataframe with all different times, then just look at the assigned column and look at max value as it will either have a max = 1 or 0

for loop through df_time_assigned to get a professor then another nested for loop to go through possible courses and keep it constant, then go through assigned variables and find if there is a 1 by taking the maximum value of assigned "list" then making constraint that X_prof_course to be that maximum value

In [90]:
#going through all profs and courses, then assigning the variable made in class_assigned to 0 or 1
#adding to model

for prof in all_profs:
    df_class_assigned_profs = df_time_assigned.loc[df_time_assigned['all_profs']==prof]
    for courses in df_class_assigned_profs['all_courses'].unique():
        df_class_assigned_profs_courses = df_class_assigned_profs.loc[df_class_assigned_profs['all_courses']==courses]
        df_class_assigned_profs_courses = df_class_assigned_profs_courses.drop(columns = ['all_courses_and_times','all_times','Day','Start_Time','End_Time','Time_Group','overlap_times','start_time_past_midnight','end_time_past_midnight'])
        mdl.add_constraint(df_class_assigned.loc[df_class_assigned['all_profs']==prof].loc[df_class_assigned['all_courses']==courses]['class_assigned'].unique()[0] == mdl.max(df_class_assigned_profs_courses['assigned']))
        

In [91]:
#creating binary variable matrix for number of preps for a professor
#0 being they don't have that many preps and 1 being they have that many preps
#only should have one 1 in each row
num_preps = ["One","Two","Three","Four"]
assigned_num_prep = mdl.binary_var_matrix(keys1 = all_profs, keys2 = num_preps, name = "%s_with_%s")
df_assigned_num_prep = DataFrame({'assigned_num_prep':assigned_num_prep})
df_assigned_num_prep.index.names = ['all_profs', 'num_preps']
df_assigned_num_prep = df_assigned_num_prep.reset_index()
#df_assigned_num_prep

In [92]:
#summing up all class_assigned variables and setting assigned_num_prep to 1 based on how many preps
#preps are summation of all class_assigned variables for a professor
for prof in all_profs:
    df_class_assigned_prof = df_class_assigned.loc[df_class_assigned['all_profs']==prof]
    mdl.add_if_then(mdl.sum(df_class_assigned_prof['class_assigned']) == 1, df_assigned_num_prep.loc[df_assigned_num_prep['all_profs']==prof].loc[df_assigned_num_prep['num_preps']=='One']['assigned_num_prep'].unique()[0] == 1)
    mdl.add_if_then(mdl.sum(df_class_assigned_prof['class_assigned']) == 2, df_assigned_num_prep.loc[df_assigned_num_prep['all_profs']==prof].loc[df_assigned_num_prep['num_preps']=='Two']['assigned_num_prep'].unique()[0] == 1)
    mdl.add_if_then(mdl.sum(df_class_assigned_prof['class_assigned']) == 3, df_assigned_num_prep.loc[df_assigned_num_prep['all_profs']==prof].loc[df_assigned_num_prep['num_preps']=='Three']['assigned_num_prep'].unique()[0] == 1)
    mdl.add_if_then(mdl.sum(df_class_assigned_prof['class_assigned']) >= 4, df_assigned_num_prep.loc[df_assigned_num_prep['all_profs']==prof].loc[df_assigned_num_prep['num_preps']=="Four"]['assigned_num_prep'].unique()[0] == 1)
    mdl.add_constraint(mdl.sum(df_assigned_num_prep.loc[df_assigned_num_prep['all_profs']==prof]['assigned_num_prep'])<=1)

In [93]:
#adding prefs to rotated df_preps_pref dataframe
df_preps_pref_melt = pd.melt(df_preps_pref, id_vars=['all_profs'], value_vars=['One', 'Two', 'Three', 'Four'], var_name = 'num_preps', value_name = 'prefs')
df_assigned_num_prep = df_assigned_num_prep.merge(df_preps_pref_melt, on = ['all_profs', 'num_preps'])

In [94]:
preps_prefs_satisfaction_series = df_assigned_num_prep['assigned_num_prep']*df_assigned_num_prep['prefs']

In [95]:
mdl.print_information()

Model: classes
 - number of variables: 56
   - binary=52, integer=0, continuous=4
 - number of constraints: 92
   - linear=50, indicator=28, equiv=14
 - parameters: defaults
 - objective: none
 - problem type is: MILP


# Preferences about back-to-back courses
## 2 sections of the same or different courses
## Change from Fall_2025_schedule.ipynb:  Now uses Theodore's code from 7 Feb 2025
### From his version of preps-and-some-b2bs-v3-Annabelle-Theodore-Brynn.ipynb

### Note that Brynn added "if time != 0" to avoid problems with asynchronous courses.  Figure out if/how to do this for this version.

July 2025:  Tried rewriting this without making the `df_time_assigned_double` copy of `df_time_assigned`, but found that this line:
`df_time_assigned = df_time_assigned.merge(df_time_assigned_small, left_on = ['all_profs', 'course_time'], right_on = ['all_profs','course_time_1'])`
caused the kernel to crash due to memory usage > 10 GB.

### Change from Feb 2025:  added option to use either the Feb_2025 version (which handles 2 back-to-back courses that are the same or different) (if b2b_Theodore_Feb_2025) or the Fall_2025_schedule version (which just handles 2 back-to-back courses that are the same)
to see if this improves memory/speed


In [96]:
if b2b_Theodore_Feb_2025:

    # start_time = timeit.default_timer()

    #This might be a poor choice for memory efficiency but I can deal with that later, right?
    df_time_assigned_double = df_time_assigned.copy()
    df_time_assigned_double.drop(columns=['Start_Time','End_Time','start_time_past_midnight','end_time_past_midnight','overlap_times','back_to_back_start_times', "Day","Time_Group"], inplace = True)
    df_time_assigned_double.rename(columns={'all_courses_and_times':'course_time_1', 'assigned':'assigned_1',},inplace=True)

    ####### Dr. Abra (10 Feb 2025) change to Theodore's code:  Remove asynchronous courses before proceeding.
    ####### Need to double-check that asynch courses still get scheduled.
    df_time_assigned_double = df_time_assigned_double.loc[df_time_assigned_double['back_to_back_end_times'].notnull()]
    #print(df_time_assigned_double)

    df_time_assigned_double = df_time_assigned_double[df_time_assigned_double['back_to_back_end_times'].map(len)>0]

    #surely this data exists somewhere already - go find it
    df_classes_and_times = pd.DataFrame( df_time_assigned.all_courses_and_times.unique(), columns = ["course_time"])
    df_classes_and_times['time_value'] = df_classes_and_times.course_time.str.split('_').str[1].astype(int) #.values()

    df_time_assigned_double = df_time_assigned_double.explode('back_to_back_end_times')

    df_time_assigned_double = df_time_assigned_double.merge(df_classes_and_times, how = 'inner', left_on = 'back_to_back_end_times', right_on = 'time_value')
    df_time_assigned_small = df_time_assigned[['all_courses_and_times','all_profs','assigned']]
    df_time_assigned_double = df_time_assigned_double.merge(df_time_assigned_small, left_on = ['all_profs', 'course_time'], right_on = ['all_profs','all_courses_and_times'])

    #This *should* be the same principle as taking a mask -- .apply works but do I need it? 
    # df_time_assigned_double['same'] = df_time_assigned_double.apply(lambda x: x.all_courses == (x.course_time.split('_')[0]),axis=1)
    df_time_assigned_double['same'] = (df_time_assigned_double['all_courses'] == (df_time_assigned_double['course_time'].str.split('_').str[0]))
    #possibly more memory efficient to not merge the 3b2b at all but I can deal with that later if it proves problematic
    df_back_to_back_pref = df_back_to_back_pref.drop('Three_b2b', axis=1) # This doesn't make much difference; reduces memory usage by the end of this cell to 3.7 GB instead of 3.8.
    df_time_assigned_double = df_time_assigned_double.merge(df_back_to_back_pref, on = 'all_profs')#.drop('Three_b2b',axis=1)

    df_time_assigned_double['b2b_pref_list_v2'] = np.nan
    # Spring_2026_sched:  on the next line, memory used goes from 1.2 to 2.9 GB
    df_time_assigned_double['b2b_pref_list_v2'] = df_time_assigned_double['b2b_pref_list_v2'].where(df_time_assigned_double['same'], (df_time_assigned_double['assigned_1'] * df_time_assigned_double['assigned'] * df_time_assigned_double['Two_same_b2b']), inplace = False)#
    # Spring_2026_sched: on the next line, memory used goes up to 3.7 or 3.8 GB
    df_time_assigned_double['b2b_pref_list_v2'] = df_time_assigned_double['b2b_pref_list_v2'].mask(df_time_assigned_double['same'], (df_time_assigned_double['assigned_1'] * df_time_assigned_double['assigned'] * df_time_assigned_double['Two_diff_b2b']), inplace = False)#
    # df_time_assigned_double = df_time_assigned_double[df_time_assigned_double['b2b_pref_list_v2'] != 0]

    # elapsed = timeit.default_timer() - start_time
    # print(elapsed)
    df_time_assigned_double #.loc[40:60] #.loc[df_time_assigned_double['end_time_1'] == 13]




In [97]:
mdl.print_information()

Model: classes
 - number of variables: 56
   - binary=52, integer=0, continuous=4
 - number of constraints: 92
   - linear=50, indicator=28, equiv=14
 - parameters: defaults
 - objective: none
 - problem type is: MILP


## Change in July 2025:
Added `if isinstance(back_to_back_time, list):` to avoid complaints about the `nan` in the back_to_back courses for Time_Index 0 (asynchronous courses).  Not sure why I didn't need this when I did the Fall 2025 schedule.

In [98]:
if b2b_Fall_2025_sched:
    #adding back-to-back pref for one course one prof
    pref_sum = 0
    b2b_pref_list = [] # append preferences to this, then concatenate it with total_satisfaction_series
    for prof in all_profs:
        df_time_assigned_by_prof = df_assigned.loc[df_assigned['all_profs']==prof]
        for course in df_time_assigned_by_prof['all_courses'].unique():
            df_time_assigned_by_prof_class = df_time_assigned_by_prof.loc[df_time_assigned_by_prof['all_courses']==course]
            df_time_assigned_by_prof_class = df_time_assigned_by_prof_class.drop(columns=['all_courses_and_times'])
            df_time_assigned_by_prof_class = pd.merge(df_time_assigned_by_prof_class, df_times)
            #print(df_time_assigned_by_prof_class)
            #am adding the preference twice, so it is half as valueable, should add an if statement to only do back to backs for larger time indices
            for time in df_time_assigned_by_prof_class['all_times']:
                #print(time)
                #print(df_time_assigned_by_prof_class.loc[df_time_assigned_by_prof_class['all_times']==time]['back_back_list'])
                df_back_to_back_comparison = df_time_assigned_by_prof_class.loc[df_time_assigned_by_prof_class['all_times']==time].reset_index()
                for back_to_back_time in df_time_assigned_by_prof_class.loc[df_time_assigned_by_prof_class['all_times']==time]['back_to_back_start_times']:
                    if isinstance(back_to_back_time, list):  ## Added in July 2025 to deal with nan for Time Index 0
                        #print(back_to_back_time)
                        df_back_to_back = df_time_assigned_by_prof_class.loc[df_time_assigned_by_prof_class['all_times'].isin(back_to_back_time)]
                        #print(df_back_to_back['assigned'].iloc[0])
                        df_back_to_back_pref_prof = df_back_to_back_pref.loc[df_back_to_back_pref['all_profs']==prof]
                        # df_back_to_back_pref_prof = df_back_to_back_pref_prof.drop(columns = ['index'])
                        df_back_to_back_pref_prof = df_back_to_back_pref_prof.reset_index()
                        int_back = df_back_to_back_pref_prof['Two_same_b2b']
                        #print(int_back[0])
                        for comp_assign in df_back_to_back_comparison['assigned']:
                            if df_back_to_back['assigned'].size > 0: # Added by Abra on 7/26/2024
                                b2b_pref_list.append(df_back_to_back['assigned'].iloc[0]*comp_assign*int_back[0])
                    

In [99]:
mdl.print_information()

Model: classes
 - number of variables: 56
   - binary=52, integer=0, continuous=4
 - number of constraints: 92
   - linear=50, indicator=28, equiv=14
 - parameters: defaults
 - objective: none
 - problem type is: MILP


# Similar Courses Preferences

In [100]:
if use_course_course_prof:
    df_course_course_similar = pd.DataFrame(columns = ['Course_1_Course_2','course_course_assigned'],index=range(df_course_course_prof_prefs.shape[0]))

    # ind is the index of the dataframe and it iterates down the rows, getting the two course preferences
    for ind in range(df_course_course_prof_prefs.shape[0]):
        df_course_course_similar_1 = df_class_assigned[df_class_assigned['all_courses'] == df_course_course_prof_prefs['Course_1'].iloc[ind]]
        df_course_course_similar_2 = df_class_assigned[df_class_assigned['all_courses'] == df_course_course_prof_prefs['Course_2'].iloc[ind]]
        df_course_course_similar.loc[ind,'Course_1_Course_2'] = "%s_%s" % (df_course_course_prof_prefs['Course_1'].iloc[ind],df_course_course_prof_prefs['Course_2'].iloc[ind])
        df_mults = df_course_course_similar_1.copy()
        df_mults = df_course_course_similar_1['class_assigned'].values*df_course_course_similar_2['class_assigned'].values
        df_course_course_similar.loc[ind,'course_course_assigned'] = df_course_course_prof_prefs['Pref'].iloc[ind]*sum(df_mults)
    # Note, Dr. Abra fixed a typo in this variable's name on 1/7/2025.
    course_course_prof_prefs_satisfaction_series = df_course_course_similar['course_course_assigned']

else:
    course_course_prof_prefs_satisfaction_series = pd.Series([])

In [101]:
mdl.print_information() # no change, good.  I *think* it would also not change even if we had use_course_course_prof = True.

Model: classes
 - number of variables: 56
   - binary=52, integer=0, continuous=4
 - number of constraints: 92
   - linear=50, indicator=28, equiv=14
 - parameters: defaults
 - objective: none
 - problem type is: MILP


# Preferences for distribution throughout day
## New in Nov 2025

In [102]:
#### TIME GROUP J #####
# Future work:  make use of preference_dist_throughtout_day_actual_Oct25.py and factor out time groups H and K similarly

#subsetting df_time_assigned so only looking at the courses that occur at time group J
df_times_J = df_time_assigned.loc[df_time_assigned['Time_Group']=='J']



# doing mod 90 so the non overlapping times will be in the same equivalence class (0,30,60)
df_times_J_mod = df_times_J.assign(start_times_mod_90 = (lambda x: x["start_time_past_midnight"]%90))
df_times_J_mod



# # doing the early/late combinations based on the "start_times_mod_90" values
df_e0_l30 = pd.concat([df_times_J_mod.loc[df_times_J_mod["start_times_mod_90"] == 0.0].loc[df_times_J_mod["end_time_past_midnight"] <= 720.0],
                     df_times_J_mod.loc[df_times_J_mod["start_times_mod_90"] == 30.0].loc[df_times_J_mod["start_time_past_midnight"] > 720.0]])

df_e0_l60 = pd.concat([df_times_J_mod.loc[df_times_J_mod["start_times_mod_90"] == 0.0].loc[df_times_J_mod["end_time_past_midnight"] <= 720.0],
                     df_times_J_mod.loc[df_times_J_mod["start_times_mod_90"] == 60.0].loc[df_times_J_mod["start_time_past_midnight"] > 720.0]])

df_e30_l0 = pd.concat([df_times_J_mod.loc[df_times_J_mod["start_times_mod_90"] == 30.0].loc[df_times_J_mod["end_time_past_midnight"] <= 720.0],
                     df_times_J_mod.loc[df_times_J_mod["start_times_mod_90"] == 0.0].loc[df_times_J_mod["start_time_past_midnight"] > 720.0]])

df_e30_l60 = pd.concat([df_times_J_mod.loc[df_times_J_mod["start_times_mod_90"] == 30.0].loc[df_times_J_mod["end_time_past_midnight"] <= 720.0],
                     df_times_J_mod.loc[df_times_J_mod["start_times_mod_90"] == 60.0].loc[df_times_J_mod["start_time_past_midnight"] > 720.0]])

df_e60_l0 = pd.concat([df_times_J_mod.loc[df_times_J_mod["start_times_mod_90"] == 60.0].loc[df_times_J_mod["end_time_past_midnight"] <= 720.0],
                     df_times_J_mod.loc[df_times_J_mod["start_times_mod_90"] == 0.0].loc[df_times_J_mod["start_time_past_midnight"] > 720.0]])

df_e60_l30 = pd.concat([df_times_J_mod.loc[df_times_J_mod["start_times_mod_90"] == 60.0].loc[df_times_J_mod["end_time_past_midnight"] <= 720.0],
                     df_times_J_mod.loc[df_times_J_mod["start_times_mod_90"] == 30.0].loc[df_times_J_mod["start_time_past_midnight"] > 720.0]])



# # putting all the about dfs into a list together
diff_dist_actual_dfs_J = [df_times_J_mod.loc[df_times_J_mod["start_times_mod_90"]==0.0],
                       df_times_J_mod.loc[df_times_J_mod["start_times_mod_90"]==30.0],
                       df_times_J_mod.loc[df_times_J_mod["start_times_mod_90"]==60.0],
                       df_e0_l30,
                       df_e0_l60,
                       df_e30_l0,
                       df_e30_l60,
                       df_e60_l0,
                       df_e60_l30]

dist_actual_time_pref_J_satisfaction_series = pd.Series()

# adding the summations of the assignment variables in those dfs in above made list so can add to satisfaction series
for i in range(0,len(diff_dist_actual_dfs_J)):
    dist_actual_time_pref_J_satisfaction_series[i] = mdl.sum(diff_dist_actual_dfs_J[i]["assigned"])

#dist_actual_time_pref_J_satisfaction_series

In [103]:
##### TIME GROUP K #####


# subsetting df_time_assigned so only looking at the courses that occur at time group K
df_times_K = df_time_assigned.loc[df_time_assigned['Time_Group']=='K']



# doing mod 120 so the non overlapping times will be in the same equivalence class (0,60)
df_times_K_mod = df_times_K.assign(start_times_mod_120 = (lambda x: x["start_time_past_midnight"]%120))



# getting rid of the WF time slot since only one meeting time on WF (8am-10am)
df_times_K_mod = df_times_K_mod.loc[df_times_K_mod["Day"] != "WF"]
df_times_K_mod



df_MW_0 = df_times_K_mod.loc[df_times_K_mod["Day"] == "MW"].loc[df_times_K_mod["start_times_mod_120"] == 0]
df_MW_60 = df_times_K_mod.loc[df_times_K_mod["Day"] == "MW"].loc[df_times_K_mod["start_times_mod_120"] == 60]
df_TR_0 = df_times_K_mod.loc[df_times_K_mod["Day"] == "TR"].loc[df_times_K_mod["start_times_mod_120"] == 0]
df_TR_60 = df_times_K_mod.loc[df_times_K_mod["Day"] == "TR"].loc[df_times_K_mod["start_times_mod_120"] == 60]



# 1
df_MW0_TR0 = pd.concat([df_times_K_mod.loc[df_times_K_mod["start_times_mod_120"]==0].loc[df_times_K_mod["Day"]=="MW"],
                         df_times_K_mod.loc[df_times_K_mod["start_times_mod_120"]==0].loc[df_times_K_mod["Day"]=="TR"]])
# 2
df_MW0_TR60 = pd.concat([df_times_K_mod.loc[df_times_K_mod["start_times_mod_120"]==0].loc[df_times_K_mod["Day"]=="MW"],
                         df_times_K_mod.loc[df_times_K_mod["start_times_mod_120"]==60].loc[df_times_K_mod["Day"]=="TR"]])
# 3
df_MW60_TR0 = pd.concat([df_times_K_mod.loc[df_times_K_mod["start_times_mod_120"]==60].loc[df_times_K_mod["Day"]=="MW"],
                         df_times_K_mod.loc[df_times_K_mod["start_times_mod_120"]==0].loc[df_times_K_mod["Day"]=="TR"]])
# 4
df_MW60_TR60 = pd.concat([df_times_K_mod.loc[df_times_K_mod["start_times_mod_120"]==60].loc[df_times_K_mod["Day"]=="MW"],
                         df_times_K_mod.loc[df_times_K_mod["start_times_mod_120"]==60].loc[df_times_K_mod["Day"]=="TR"]])



df_1e_2l = pd.concat([df_MW0_TR0.loc[df_MW0_TR0["end_time_past_midnight"] <= 720.0],
                      df_MW0_TR60.loc[df_MW0_TR60["start_time_past_midnight"] > 720.0]])

df_1e_3l = pd.concat([df_MW0_TR0.loc[df_MW0_TR0["end_time_past_midnight"] <= 720.0],
                      df_MW60_TR0.loc[df_MW60_TR0["start_time_past_midnight"] > 720.0]])

df_1e_4l = pd.concat([df_MW0_TR0.loc[df_MW0_TR0["end_time_past_midnight"] <= 720.0],
                      df_MW60_TR60.loc[df_MW60_TR60["start_time_past_midnight"] > 720.0]])

df_2e_3l = pd.concat([df_MW0_TR60.loc[df_MW0_TR60["end_time_past_midnight"] <= 720.0],
                      df_MW60_TR0.loc[df_MW60_TR0["start_time_past_midnight"] > 720.0]])

df_2e_4l = pd.concat([df_MW0_TR60.loc[df_MW0_TR60["end_time_past_midnight"] <= 720.0],
                      df_MW60_TR60.loc[df_MW60_TR60["start_time_past_midnight"] > 720.0]])

df_3e_4l = pd.concat([df_MW60_TR0.loc[df_MW60_TR0["end_time_past_midnight"] <= 720.0],
                      df_MW60_TR60.loc[df_MW60_TR60["start_time_past_midnight"] > 720.0]])




# # putting all the about dfs into a list together
diff_dist_actual_dfs_K = [df_MW0_TR0,
                          df_MW0_TR60,
                          df_MW60_TR0,
                          df_MW60_TR60,
                          df_1e_2l,
                          df_1e_3l,
                          df_1e_4l,
                          df_2e_3l,
                          df_2e_4l,
                          df_3e_4l                          
                          ]


dist_actual_time_pref_K_satisfaction_series = pd.Series(dtype='object')

# # #adding the summations of the assignment variables in those dfs in above made list so can add to satisfaction series
for i in range(0,len(diff_dist_actual_dfs_K)):
    dist_actual_time_pref_K_satisfaction_series[i] = mdl.sum(diff_dist_actual_dfs_K[i]["assigned"])

dist_actual_time_pref_K_satisfaction_series


0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
dtype: object

In [104]:
##### TIME GROUP H ######


# subsetting df_time_assigned so only looking at the courses that occur at time group H
df_times_H = df_time_assigned.loc[df_time_assigned['Time_Group']=='H']
df_times_H


group_H_assigned = mdl.binary_var_matrix(keys1=df_times_H['all_courses'].unique(),keys2=df_times_H["Start_Time"].unique(),name="%s_at_time_%s")
df_group_H_assigned = DataFrame({'H_course_time_assigned':group_H_assigned})
df_group_H_assigned.index.names = ['all_courses', 'Start_Time']
df_group_H_assigned = df_group_H_assigned.reset_index()




for course in df_times_H['all_courses'].unique():
    df_H_assigned_course = df_times_H.loc[df_times_H['all_courses'] == course]
    time_options = df_time_groups[df_time_groups["Class"]==course]['Time_Group']
    for time in df_times_H.loc[df_times_H['all_courses']==course]['Start_Time'].unique():
        df_H_course_time = df_H_assigned_course.loc[df_H_assigned_course['Start_Time'] == time]
        mdl.add_constraint(df_group_H_assigned.loc[df_group_H_assigned['all_courses']==course].loc[df_group_H_assigned['Start_Time']==time]['H_course_time_assigned'].unique()[0] == mdl.max(df_H_course_time['assigned']))
df_group_H_assigned




sum_dist_H_assigned = mdl.integer_var_list(keys=all_courses,name='%s_at_all_times_group_H')



satisfaction_series_H_assigned = DataFrame({'all_courses':all_courses,'sum_times_assigned_H':sum_dist_H_assigned})

for course in all_courses:
    mdl.add_constraint(satisfaction_series_H_assigned.loc[satisfaction_series_H_assigned['all_courses']==course]['sum_times_assigned_H'].unique()[0] == mdl.sum(df_group_H_assigned.loc[df_group_H_assigned['all_courses']==course]['H_course_time_assigned']))
    
satisfaction_series_H_assigned



dist_actual_time_pref_H_satisfaction_series = pd.Series(satisfaction_series_H_assigned["sum_times_assigned_H"])

# adding the objective to the model
### Small change in July 2025:
Using different versions of satisfaction series depending on which version of b2b code we're using

In [105]:
### Satisfaction Elements are in a later code cell
if b2b_Theodore_Feb_2025:
    satisfaction_elements = pd.concat([class_prefs_satisfaction_series,
                                       time_prefs_satisfaction_series, 
                                       preps_prefs_satisfaction_series, 
                                       #pd.Series(course_course_time_pref_series), 
                                       pd.Series(course_time_pref_series),
                                       course_course_prof_prefs_satisfaction_series, 
                                       pd.Series(df_time_assigned_double['b2b_pref_list_v2']),
                                      dist_actual_time_pref_H_satisfaction_series,
                                      dist_actual_time_pref_J_satisfaction_series,
                                      dist_actual_time_pref_K_satisfaction_series]) 

if b2b_Fall_2025_sched:
    satisfaction_elements = pd.concat([class_prefs_satisfaction_series,
                                       time_prefs_satisfaction_series, 
                                       preps_prefs_satisfaction_series, 
                                       #pd.Series(course_course_time_pref_series), 
                                       pd.Series(course_time_pref_series),
                                       course_course_prof_prefs_satisfaction_series, 
                                       pd.Series(b2b_pref_list),
                                      dist_actual_time_pref_H_satisfaction_series,
                                      dist_actual_time_pref_J_satisfaction_series,
                                      dist_actual_time_pref_K_satisfaction_series])


In [106]:
total_satisfaction = mdl.sum(satisfaction_elements) 

In [107]:
mdl.maximize(total_satisfaction)

# Solve model

In [108]:
mdl.print_information()

Model: classes
 - number of variables: 78
   - binary=67, integer=2, continuous=9
 - number of constraints: 124
   - linear=72, indicator=38, equiv=14
 - parameters: defaults
 - objective: maximize quadratic
 - problem type is: MIQP


In [109]:
# mdl.get_objective_expr()
# mdl.lp_string

In [110]:
# New in Nov 2025
if listen_progress:
    mdl.add_progress_listener(TextProgressListener())

In [111]:
solution = mdl.solve(clean_before_solve=False) #clean_before_solve = True, will restart the model, whereas false (default) will pick up where last model left off
mdl.report()

  1+: Node=0 Left=1 Best Integer=58.5000, Best Bound=78.5000, gap=34.19%, ItCnt=30 [0.0s]
  2+: Node=0 Left=1 Best Integer=75.0000, Best Bound=78.5000, gap=4.67%, ItCnt=30 [0.0s]
* model classes solved with objective = 78.000


In [112]:
solve_status = mdl.get_solve_status()
print(solve_status)
print(mdl.solve_details.time) # is this in seconds? Seems like it.

if solve_status.name == 'INFEASIBLE_SOLUTION':
    infeasible = True
else:
    infeasible = False

JobSolveStatus.OPTIMAL_SOLUTION
32.96110010147095


# Relax Solution, if necessary
TPS 11/19/24

In [113]:
if infeasible:

    cr = ConflictRefiner()
    #type(cr)
    conflicting_constraints = cr.refine_conflict(mdl, display = True)
    conflicting_constraints.display()

In [114]:
if infeasible:

    rx = Relaxer()
    #type(rx)
    relaxed_solution = rx.relax(mdl)
    rx.print_information()
    relaxed_solution.display()

In [115]:
if infeasible:
    df_solution = relaxed_solution.as_df()
else:
    df_solution = solution.as_df()
df_solution

,name,value
0,Math 347_1_with_prof_ProfA,1.000000e+00
1,Math 246_2_with_prof_ProfA,1.000000e+00
2,Math 246_3_with_prof_ProfA,1.000000e+00
3,Math 246_4_with_prof_ProfB,1.000000e+00
4,Math 246_5_with_prof_ProfB,1.000000e+00
5,Math 246_6_with_prof_ProfB,1.000000e+00
6,_bool#1,1.000000e+00
7,_bool#4,1.000000e+00
8,_bool#6,1.000000e+00
9,ProfA_at_Math 246,1.000000e+00


In [116]:
# Drop placeholder "_bool" variables
solution=df_solution.astype(str)
solution=solution.iloc[[True if (type(solution.iloc[a]['name']) is str and not solution.iloc[a]['name'].startswith("_bool")) else False for a in range(solution.shape[0])]]
solution

,name,value
0,Math 347_1_with_prof_ProfA,1.0
1,Math 246_2_with_prof_ProfA,1.0
2,Math 246_3_with_prof_ProfA,1.0
3,Math 246_4_with_prof_ProfB,1.0
4,Math 246_5_with_prof_ProfB,1.0
5,Math 246_6_with_prof_ProfB,1.0
9,ProfA_at_Math 246,1.0
10,ProfA_at_Math 347,1.0
11,ProfB_at_Math 246,1.0
12,x23,1.0


In [117]:
# pd.concat([solution, total])
mid_df= solution.merge(df_assigned.astype(str), how="left", left_on="name", right_on="assigned")
mid_df = mid_df.loc[mid_df["all_courses"].notnull()]
# new_df=new_df.loc[new_df.value==1]
mid_df['all_times'] = mid_df['all_times'].astype(int)
#mid_df.to_csv("test.csv", index=False) # for debugging the extra row
mid_df

,name,value,all_courses,all_profs,all_times,all_courses_and_times,assigned
0,Math 347_1_with_prof_ProfA,1.0,Math 347,ProfA,1,Math 347_1,Math 347_1_with_prof_ProfA
1,Math 246_2_with_prof_ProfA,1.0,Math 246,ProfA,2,Math 246_2,Math 246_2_with_prof_ProfA
2,Math 246_3_with_prof_ProfA,1.0,Math 246,ProfA,3,Math 246_3,Math 246_3_with_prof_ProfA
3,Math 246_4_with_prof_ProfB,1.0,Math 246,ProfB,4,Math 246_4,Math 246_4_with_prof_ProfB
4,Math 246_5_with_prof_ProfB,1.0,Math 246,ProfB,5,Math 246_5,Math 246_5_with_prof_ProfB
5,Math 246_6_with_prof_ProfB,1.0,Math 246,ProfB,6,Math 246_6,Math 246_6_with_prof_ProfB


In [118]:
# This code cell will add a row for the lab section of Math 347, if it's being offered.
new_df = mid_df.merge(df_times, on =['all_times'])
shifts = new_df.reset_index().set_index(['Start_Time',"End_Time","Day"]).index.values
new_df['shift'] = shifts
new_df = new_df.merge(df_credits, on = ['all_courses'])
# new_df.drop(columns=['Notes'], inplace=True)
new_df

,name,value,all_courses,all_profs,all_times,all_courses_and_times,assigned,Day,Start_Time,End_Time,Time_Group,start_time_past_midnight,end_time_past_midnight,overlap_times,back_to_back_start_times,back_to_back_end_times,shift,number_of_credits
0,Math 347_1_with_prof_ProfA,1.0,Math 347,ProfA,1,Math 347_1,Math 347_1_with_prof_ProfA,MWF,01:00:00,01:50:00,L,780,830,"[1, 4, 5]",[],[6],"(01:00:00, 01:50:00, MWF)",4
1,Math 347_1_with_prof_ProfA,1.0,Math 347,ProfA,1,Math 347_1,Math 347_1_with_prof_ProfA,R,01:00:00,02:50:00,L,780,890,"[1, 4, 5]",[],[6],"(01:00:00, 02:50:00, R)",4
2,Math 246_2_with_prof_ProfA,1.0,Math 246,ProfA,2,Math 246_2,Math 246_2_with_prof_ProfA,MWRF,08:00:00,08:50:00,H,480,530,[2],[],[3],"(08:00:00, 08:50:00, MWRF)",4
3,Math 246_3_with_prof_ProfA,1.0,Math 246,ProfA,3,Math 246_3,Math 246_3_with_prof_ProfA,MWRF,09:00:00,09:50:00,H,540,590,[3],[2],[],"(09:00:00, 09:50:00, MWRF)",4
4,Math 246_4_with_prof_ProfB,1.0,Math 246,ProfB,4,Math 246_4,Math 246_4_with_prof_ProfB,MWRF,01:00:00,01:50:00,H,780,830,"[1, 4]",[],[5],"(01:00:00, 01:50:00, MWRF)",4
5,Math 246_5_with_prof_ProfB,1.0,Math 246,ProfB,5,Math 246_5,Math 246_5_with_prof_ProfB,MWRF,02:00:00,02:50:00,H,840,890,"[1, 5]","[1, 4]",[6],"(02:00:00, 02:50:00, MWRF)",4
6,Math 246_6_with_prof_ProfB,1.0,Math 246,ProfB,6,Math 246_6,Math 246_6_with_prof_ProfB,MWRF,03:00:00,03:50:00,H,900,950,[6],"[1, 5]",[],"(03:00:00, 03:50:00, MWRF)",4


In [119]:
# Get rid of columns with "value" close to 0 (they should be exactly 0)
# To do this, we first need to convert the value from string to numeric
# Under what circumstances do we get values close to 0?
new_df["num_value"] = pd.to_numeric(new_df.value)
new_df = new_df[new_df.num_value > 0.8]
new_df

,name,value,all_courses,all_profs,all_times,all_courses_and_times,assigned,Day,Start_Time,End_Time,Time_Group,start_time_past_midnight,end_time_past_midnight,overlap_times,back_to_back_start_times,back_to_back_end_times,shift,number_of_credits,num_value
0,Math 347_1_with_prof_ProfA,1.0,Math 347,ProfA,1,Math 347_1,Math 347_1_with_prof_ProfA,MWF,01:00:00,01:50:00,L,780,830,"[1, 4, 5]",[],[6],"(01:00:00, 01:50:00, MWF)",4,1.0
1,Math 347_1_with_prof_ProfA,1.0,Math 347,ProfA,1,Math 347_1,Math 347_1_with_prof_ProfA,R,01:00:00,02:50:00,L,780,890,"[1, 4, 5]",[],[6],"(01:00:00, 02:50:00, R)",4,1.0
2,Math 246_2_with_prof_ProfA,1.0,Math 246,ProfA,2,Math 246_2,Math 246_2_with_prof_ProfA,MWRF,08:00:00,08:50:00,H,480,530,[2],[],[3],"(08:00:00, 08:50:00, MWRF)",4,1.0
3,Math 246_3_with_prof_ProfA,1.0,Math 246,ProfA,3,Math 246_3,Math 246_3_with_prof_ProfA,MWRF,09:00:00,09:50:00,H,540,590,[3],[2],[],"(09:00:00, 09:50:00, MWRF)",4,1.0
4,Math 246_4_with_prof_ProfB,1.0,Math 246,ProfB,4,Math 246_4,Math 246_4_with_prof_ProfB,MWRF,01:00:00,01:50:00,H,780,830,"[1, 4]",[],[5],"(01:00:00, 01:50:00, MWRF)",4,1.0
5,Math 246_5_with_prof_ProfB,1.0,Math 246,ProfB,5,Math 246_5,Math 246_5_with_prof_ProfB,MWRF,02:00:00,02:50:00,H,840,890,"[1, 5]","[1, 4]",[6],"(02:00:00, 02:50:00, MWRF)",4,1.0
6,Math 246_6_with_prof_ProfB,1.0,Math 246,ProfB,6,Math 246_6,Math 246_6_with_prof_ProfB,MWRF,03:00:00,03:50:00,H,900,950,[6],"[1, 5]",[],"(03:00:00, 03:50:00, MWRF)",4,1.0


In [120]:
# This version is for graphing
df_schedule=new_df.filter(["all_profs","all_courses","number_of_credits", "start_time_past_midnight", "end_time_past_midnight", "Day"])

# This version is for exporting to csv
df_schedule_to_export=new_df.filter(["all_profs","all_courses","number_of_credits", "Day", "Start_Time", "End_Time", "all_times"])

### Small change from Feb 2025:
Changed from class_file_short to qualtrics_file_short

In [121]:
now = datetime.now()
today_str = now.strftime("%Y:%m:%d_%H:%M:%S")
output_file_name = f'results_{qualtrics_file_short}_{today_str}.csv'

df_schedule_to_export.to_csv(output_file_name, index=False)